In [1]:
########## organize thes summary files

import glob
import h5py

fnames = glob.glob('*/*/summ.hdf')

files = {}
for fname in fnames:
    net, time, _ = fname.split('/')
    time = int(time)
    
    if net not in files:
        files[net] = {}
        
    files[net][time] = fname        

In [2]:
######### determine the weights from uniform in dL to uniform in comoving volume 

from pycbc.cosmology import cosmological_quantity_from_redshift, redshift
from numpy.random import uniform
from scipy.stats import gaussian_kde
import pylab
import numpy

inj = h5py.File('./injection.hdf', 'r')
idist = inj['distance'][:]

# crude finite differencing is sufficient here
cp = cosmological_quantity_from_redshift(redshift(idist + 0.01), 'comoving_volume')
cm = cosmological_quantity_from_redshift(redshift(idist - 0.01), 'comoving_volume')
w = cp - cm
w /= w.sum()

total = cosmological_quantity_from_redshift(redshift(idist.max()), 'comoving_volume')
print(idist.max())
rate = total / 1e9 * 300 # Sets the assumed rate per volume
w *= rate

19999.946205176246


In [5]:
##### Oranize retrieval of summary data and allow for masking / subselection based on criteria (such as distance)

asizes = [0.1, 1, 10, 100, 1000, 44000]
alabel = ['A$_{90} <$ 0.1 deg$^2$', 
          'A$_{90} <$ 1 deg$^2$',
          'A$_{90} <$ 10 deg$^2$',
          'A$_{90} <$ 100 deg$^2$',
          'A$_{90} <$ 1000 deg$^2$',
          'All Sky']

def process(select):
    a90 = {}
    d90 = {}
    times = {}
    for net in files:
        for time in files[net]:
            fname = files[net][time]
            f = h5py.File(fname, 'r')

            snr = f['snr'][:]
            k = snr > 12

            if net not in times:
                times[net] = []
            times[net].append(time)

            if net not in a90:
                a90[net] = {}
            for s in asizes:
                if s not in a90[net]:
                    a90[net][s] = []
                
                ak =  f['a90'][:] < s
                wf = w[k & ak & select]
                e = ((wf ** 2.0).sum() / len(wf) - (wf.sum() / len(wf)) ** 2.0)
                e = (len(wf) * e) ** 0.5
                a90_num = wf.sum() 
                
                
                ds = idist[k & ak & select]
                l = ds.argsort()
                ds = ds[l]
                wcount = wf[l].cumsum() / wf.sum()

                if len(wcount) < 2:
                    dmax = 0
                else:
                    lmax = numpy.searchsorted(wcount, 0.9)
                    dmax = ds[lmax]
                
                a90[net][s].append((a90_num, e, dmax)) 

    for k in a90:
        times[k] = numpy.array(times[k])
        l = times[k].argsort()
        times[k] = times[k][l]
        for k2 in asizes:
            a90[k][k2] = numpy.array(a90[k][k2])[l]
    return times, a90

In [ ]:
import warnings
warnings.filterwarnings('ignore')

netname = {'CE1':("C$_{1}^U$", 'solid', 's', 'red'),
           'CE2':("C$_{2}^U$", 'solid', 's', 'blue'),
           'ET_5':("E$_{5}$", 'solid', 's', 'green'),
           'ET':('E', 'solid', 's', 'purple'),
           'CE1_LIGO':('C$_{1}^U$HLI', 'dotted', 'H', 'black'),
           'CE2_LIGO':('C$_{2}^U$HLI', 'dotted', 'H', 'grey'),
           'CE2_REST':("C$_{2}^U$HLVKI", 'dotted', 'H', 'yellow'),
           'CE1_CE1':('C$_{1}^U$C$_{1}^A$', 'solid', 'D', 'orange'),
           'CE2_CE2':('C$_{2}^U$C$_{2}^A$', 'solid', 'D', 'darkred'),
           'CE1_ET':("C$_{1}^U$E", 'solid', 'D', 'aqua'),
           'CE2_ET':("C$_{2}^U$E", 'solid', 'D', 'magenta'),
           'CE1_CE1_ET':("C$_{1}^U$C$_{1}^A$E", 'solid', 'D', 'palegreen'),
           'CE2_CE2_ET':("C$_{2}^U$C$_{2}^A$E", 'solid', 'D', 'peru'),
           'ligo_3g':('C$_{2}^U$C$_{2}^A$EHLI', 'dotted', 'H', 'tan'),
           'all':('C$_{2}^U$C$_{2}^A$EHLVKI', 'dotted', 'H', 'darkblue'),
          }

pylab.rc('text', usetex=True)
for dlimit in [20000, 1000, 500]:
    f, axs = pylab.subplots(2, 3, figsize=[9.5, 11], dpi=200, sharex=True, sharey='row')

    total = cosmological_quantity_from_redshift(redshift(dlimit), 'comoving_volume')
    rate = total / 1e9 * 300
    print(rate)
    
    for j, (s, aname) in enumerate(zip(asizes, alabel)):
        ps = []
        ls = []

        pylab.sca(axs[j // 3][j % 3])

        if dlimit < 20000:
            paname = aname + ', d$_L < {}$ Mpc'.format(dlimit)
        else:
            paname = aname + ', d$_L < 20$ Gpc'.format(dlimit)
        
        pylab.text(0.1, 0.95, paname,
         horizontalalignment='left',
         verticalalignment='center',
         backgroundcolor='white',
         transform = pylab.gca().transAxes)

        select = idist < dlimit
        times, a90 = process(select)
        for i, net in enumerate(list(netname.keys())):
                label = net if j == 0 else None

                if net in netname:
                    label, lstyle, marker, color = netname[net]

                val = a90[net][s][:,0]
                e = a90[net][s][:,1]

                pylab.axhspan(rate, 1e5, linestyle='--', linewidth=1, color='grey')
                l = pylab.errorbar(times[net] / 60.0, val, yerr=e, marker=marker,
                                label=label, c=color, linestyle=lstyle, alpha=0.9)      

        if j == 5:
            pylab.legend(title='Detector Network', ncol=2, fontsize=9)

        pylab.yscale('log')
        pylab.xscale('log')
        pylab.xlim(xmin=35, xmax=0.9)

        if j >= 3:
            pylab.ylim(ymin=0.1, ymax=1e5 if rate > 1e5 else rate * 2.5)
        else:
            pylab.ylim(ymin=0.1, ymax=1e3 if rate > 1e3 else rate * 2.5)

        pylab.grid(which='major', color='darkgray')
        pylab.grid(which='minor', linewidth=0.4, color='lightgray')

        if j == 0 or j == 3:
            pylab.ylabel('Observations [$\\textrm{yr}^-1$]', fontsize=10)
        if j == 4:
            pylab.xlabel('Time to Merger [Minutes]', fontsize=10)

    pylab.tight_layout()
    pylab.subplots_adjust(wspace=.04, hspace=.05)
    pylab.savefig('area-{}.pdf'.format(dlimit))

253381.63546944404
730.9168746906909
